In [1]:
import nbaPlayerStatModeling as nbaModel

In [2]:
nba = nbaModel.playerStatModel(
        day_offset = 0, 
        season = '2024-25', 
        perMode = 'PerGame',
        num_simulations = 10000
)

In [ ]:
nba.get_teams_playing()
nba.get_teams_data(
    base_url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType={stattype}&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
)
nba.get_players_playing(
    url_base_nba_player_stat = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID={tid}&TwoWay=0&VsConference=&VsDivision=&Weight=',
    minute_cutoff = 0
)
nba.get_props()

10  games today...


In [8]:
#nba.df_players = nba.df_players.iloc[[1,10,19,25,30], :]
nba.get_player_home_adv(
    use_default=False
)

1 / 147 player splits..
2 / 147 player splits..
3 / 147 player splits..
4 / 147 player splits..
5 / 147 player splits..
6 / 147 player splits..
7 / 147 player splits..
8 / 147 player splits..
9 / 147 player splits..
10 / 147 player splits..
11 / 147 player splits..
12 / 147 player splits..
13 / 147 player splits..
14 / 147 player splits..
15 / 147 player splits..
16 / 147 player splits..
17 / 147 player splits..
18 / 147 player splits..
19 / 147 player splits..
20 / 147 player splits..
21 / 147 player splits..
22 / 147 player splits..
23 / 147 player splits..
24 / 147 player splits..
25 / 147 player splits..
26 / 147 player splits..
27 / 147 player splits..
28 / 147 player splits..
29 / 147 player splits..
30 / 147 player splits..
31 / 147 player splits..
32 / 147 player splits..
33 / 147 player splits..
34 / 147 player splits..
35 / 147 player splits..
36 / 147 player splits..
37 / 147 player splits..
38 / 147 player splits..
39 / 147 player splits..
40 / 147 player splits..
41 / 147 

In [73]:
#df_copy = nba.df_players.copy()
nba.df_players = df_copy.copy()

In [74]:
nba.model_expected_minutes()
nba.calculate_model_inputs()
nba.df_players = nba.df_players.fillna(0)
nba.model_expected_stats_poisson()

In [75]:
nba.num_simulations = 10
nba.model_expected_pts()

In [76]:
nba.df_players

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,expPts,REBoProb,REBoOdds,REBoOdds_deci,ASToProb,ASToOdds,ASToOdds_deci,PTSoProb,PTSoOdds,PTSoOdds_deci
0,1627747,Caris LeVert,1610612737,ATL,23.7,4.0,8.5,0.471,2.0,5.5,...,13.645484,0.057351,1643.641576,17.436,0.364048,174.689112,2.747,0.8,-400.000000,1.250
1,1627747,Caris LeVert,1610612737,ATL,23.8,3.6,7.8,0.453,1.6,3.9,...,10.382429,0.224267,345.896955,4.459,0.060090,1564.165646,16.642,0.6,-150.000000,1.667
2,203991,Clint Capela,1610612737,ATL,22.4,4.2,7.3,0.573,0.0,0.0,...,8.684608,0.154146,548.736501,6.487,0.471213,112.218423,2.122,0.7,-233.333333,1.429
3,1629631,De'Andre Hunter,1610612739,CLE,28.7,6.1,13.3,0.461,2.6,6.7,...,18.527812,0.180302,454.625814,5.546,0.299970,233.366948,3.334,0.5,-100.000000,2.000
4,1629631,De'Andre Hunter,1610612739,CLE,23.0,3.0,7.0,0.429,3.0,4.0,...,11.690885,0.042276,2265.391873,23.654,0.000000,NaN,NaN,0.4,150.000000,2.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,1630182,Josh Green,1610612766,CHA,28.6,2.6,6.0,0.437,1.4,3.5,...,9.925178,0.220702,353.099372,4.531,0.285226,250.599586,3.506,0.2,400.000000,5.000
154,1630163,LaMelo Ball,1610612766,CHA,32.7,9.5,22.8,0.416,4.0,11.9,...,26.101839,0.106946,835.055036,9.351,0.148843,571.848700,6.718,0.5,-100.000000,2.000
155,1628970,Miles Bridges,1610612766,CHA,32.0,7.2,16.4,0.438,2.0,6.7,...,22.180591,0.131346,661.345720,7.613,0.189576,427.493923,5.275,0.5,-100.000000,2.000
156,1631217,Moussa Diabaté,1610612766,CHA,18.9,2.1,3.4,0.616,0.0,0.0,...,5.981785,0.131436,660.824099,7.608,0.350779,185.079475,2.851,0.2,400.000000,5.000


In [74]:
nba.df_players[['PLAYER_NAME', 'OREB', 'DREB', 'REB', 'reb_line', 'expReb']]

,PLAYER_NAME,OREB,DREB,REB,reb_line,expReb
0,Cade Cunningham,0.9,5.5,6.4,5.5,7.362742
1,Paul George,0.6,4.7,5.3,5.5,6.828345
2,Dillon Brooks,1.2,2.6,3.9,3.5,4.402719
3,Gary Trent Jr.,0.2,2.2,2.4,2.5,3.117286
4,Ja'Kobe Walter,0.7,1.7,2.4,0.0,2.683143


In [56]:
for i in nba.df_players.columns: print(i)

PLAYER_ID
PLAYER_NAME
TEAM_ID
TEAM_ABBREVIATION
MIN
FGM
FGA
FG_PCT
FG3M
FG3A
FG3_PCT
FTM
FTA
FT_PCT
OREB
DREB
REB
AST
TOV
STL
BLK
BLKA
PF
PFD
PTS
PLUS_MINUS
homeTeam
oppTeamId
PACE
OREB_PCT
DREB_PCT
oppPACE
oppPts
oppAst
PACEadj
OREBadj
DREBadj
PTSadj
ASTadj
threes_line
stl_line
reb_line
ra_line
pts_line
pra_line
pr_line
pa_line
blk_line
ast_line
sb_line
threes_oOdds
stl_oOdds
reb_oOdds
ra_oOdds
pts_oOdds
pra_oOdds
pr_oOdds
pa_oOdds
blk_oOdds
ast_oOdds
sb_oOdds
threes_uOdds
stl_uOdds
reb_uOdds
ra_uOdds
pts_uOdds
pra_uOdds
pr_uOdds
pa_uOdds
blk_uOdds
ast_uOdds
sb_uOdds
MIN_HOMEadj
FGM_HOMEadj
FGA_HOMEadj
FG3M_HOMEadj
FG3A_HOMEadj
FTM_HOMEadj
FTA_HOMEadj
OREB_HOMEadj
DREB_HOMEadj
REB_HOMEadj
AST_HOMEadj
TOV_HOMEadj
STL_HOMEadj
BLK_HOMEadj
BLKA_HOMEadj
PTS_HOMEadj
MIN_ROADadj
FGM_ROADadj
FGA_ROADadj
FG3M_ROADadj
FG3A_ROADadj
FTM_ROADadj
FTA_ROADadj
OREB_ROADadj
DREB_ROADadj
REB_ROADadj
AST_ROADadj
TOV_ROADadj
STL_ROADadj
BLK_ROADadj
BLKA_ROADadj
PTS_ROADadj
MINadj
FG2M
FG2A
FG2_PCT
FGA_FT

In [30]:
nba.df_players[['PLAYER_NAME','homeTeam', 'MINadj','PACEadj', 
                'OREB', 'OREBadj', 'OREB_HOMEadj',  'DREB', 'DREBadj','DREB_HOMEadj', 'expReb', 
                'AST', 'ASTadj', 'expAst', 
                'PTS', 'PTSadj', 'expPts']]

,PLAYER_NAME,homeTeam,MINadj,PACEadj,OREB,OREBadj,OREB_HOMEadj,DREB,DREBadj,DREB_HOMEadj,expReb,AST,ASTadj,expAst,PTS,PTSadj,expPts
0,Cade Cunningham,1,1,1.007106,0.9,0.982652,0.535714,5.5,0.965917,1.286957,7.362742,9.5,0.999748,10.442623,25.5,0.991145,25.262834
1,Paul George,0,1,0.986124,0.6,0.938142,0.857143,4.7,1.071984,1.391304,6.828345,4.5,0.980813,5.090772,16.6,0.996440,19.259470
2,Dillon Brooks,1,1,0.993778,1.2,1.061402,1.251553,2.6,0.980059,1.113043,4.402719,1.5,0.992174,1.205333,13.6,1.028213,12.550923
3,Gary Trent Jr.,1,1,1.014071,0.2,1.075097,1.254545,2.2,1.035214,1.231313,3.117286,1.1,1.045191,1.371242,10.2,0.999971,9.624349
4,Ja'Kobe Walter,0,1,1.006261,0.7,1.016891,1.578947,1.7,0.900863,1.118421,2.683143,1.4,0.855844,1.410790,7.8,0.959372,10.210140


In [63]:
import requests
url_base_nba_player_stat = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID={tid}&TwoWay=0&VsConference=&VsDivision=&Weight='
minute_cutoff = 15.0

team_id = 1610612737
player_info_url = url_base_nba_player_stat.format(
    per = 'PerGame',
    sid = '2024-25', 
    tid = team_id
) 
response = requests.get(url=player_info_url, headers=nba.HEADERS).json()
columns_player_stats = response['resultSets'][0]['headers']

for j in response['resultSets'][0]['rowSet']:
    print(j)

[203992, 'Bogdan Bogdanović', 'Bogdan', 1610612746, 'LAC', 32.0, 24, 12, 12, 0.5, 24.9, 3.5, 9.3, 0.371, 1.8, 6.1, 0.301, 1.3, 1.4, 0.882, 0.5, 2.3, 2.8, 2.0, 1.5, 0.8, 0.3, 0.2, 2.4, 1.6, 10.0, -1.8, 18.1, 0, 0, 18.8, 11, 10, 13, 6, 7, 11, 6, 20, 6, 3, 16, 10, 13, 3, 11, 12, 12, 7, 5, 8, 13, 14, 6, 8, 10, 14, 12, 10, 2, 10]
[1627747, 'Caris LeVert', 'Caris', 1610612737, 'ATL', 30.0, 2, 2, 0, 1.0, 23.7, 4.0, 8.5, 0.471, 2.0, 5.5, 0.364, 3.5, 4.0, 0.875, 0.0, 5.5, 5.5, 1.0, 0.5, 0.0, 0.0, 0.5, 3.5, 2.0, 13.5, -0.5, 21.1, 0, 0, 22.0, 17, 16, 18, 1, 9, 8, 7, 8, 5, 4, 9, 3, 3, 4, 20, 3, 4, 17, 16, 18, 17, 9, 1, 5, 5, 7, 8, 10, 2, 8]
[203991, 'Clint Capela', 'Clint', 1610612737, 'ATL', 30.0, 45, 22, 23, 0.489, 22.4, 4.2, 7.3, 0.573, 0.0, 0.0, 0.0, 1.0, 1.9, 0.541, 3.3, 5.7, 9.0, 1.2, 1.0, 0.6, 1.0, 0.7, 2.0, 1.8, 9.4, -0.8, 25.8, 11, 0, 22.8, 5, 4, 5, 10, 10, 7, 10, 3, 20, 20, 20, 12, 8, 18, 1, 2, 2, 15, 10, 14, 1, 6, 7, 7, 11, 9, 6, 4, 2, 7]
[1631342, 'Daeqwon Plowden', 'Daeqwon', 16106127

In [ ]:
c = nba.df_players.copy()
c['TEAM_ABBREVIATION'].value_counts()[:1].index.tolist()

['BOS']

In [ ]:
c['TEAM_ABBREVIATION'][9:29]

7     ATL
8     ATL
9     ATL
10    ATL
11    ATL
12    ATL
13    ATL
14    BOS
15    BOS
16    BOS
17    BOS
18    BOS
19    BOS
20    BOS
21    CLE
22    CLE
23    CLE
24    CLE
25    CLE
26    CLE
27    CLE
28    NOP
Name: TEAM_ABBREVIATION, dtype: object